In [1]:
import random
import hashlib
from cryptography.fernet import Fernet
import numpy as np
import random
import pandas as pd
from collections import Counter

In [2]:
data = pd.read_csv('sample_data.csv')
data

,Zip,Age,Education,Martial Status,Occupation,Race,Sex
0,64152,39,Bachelors,Never-married,Adm-clerical,White,Male
1,61523,50,Bachelors,Married-civ-spouse,Exec-managerial,White,Male
2,95668,38,HS-grad,Divorced,Handlers-cleaners,White,Male
3,25503,53,11th,Married-civ-spouse,Handlers-cleaners,Black,Male
4,75387,28,Bachelors,Married-civ-spouse,Prof-specialty,Black,Female
...,...,...,...,...,...,...,...
58,49792,41,HS-grad,Married-civ-spouse,Adm-clerical,White,Male
59,43800,30,HS-grad,Married-civ-spouse,Machine-op-inspct,White,Male
60,54588,30,Bachelors,Married-civ-spouse,Sales,White,Male
61,21928,32,7th-8th,Married-spouse-absent,Sales,White,Male


In [3]:
def get_zip_partition(zipcode):
    if zipcode < 30000: return 1
    if zipcode < 50000: return 2
    return 3
def get_age_partition(age):
    if age < 30: return 4
    if age < 50: return 1
    if age < 70: return 3
    return 2

In [4]:
encryption_key = Fernet.generate_key()
fernet_f = Fernet(encryption_key)
encrypted_tuples = {}
for index, row in data.iterrows():
    str_row = ";; ".join([str(t) for t in tuple(row)])
    encrypted_tuples[index] = [fernet_f.encrypt(str_row.encode('utf-8')), 
                                    get_zip_partition(row['Zip']), 
                                    get_age_partition(row['Age'])]
encrypted_data = pd.DataFrame.from_dict(encrypted_tuples, orient='index', columns = ["EncryptedTuple", "ZipP", "AgeP"])
encrypted_data

,EncryptedTuple,ZipP,AgeP
0,b'gAAAAABgSoAe6tcU3Aufc0uozxKkSGsgbw3DTl3C2wvX...,3,1
1,b'gAAAAABgSoAe1mNtUDivVAnmJvVEr_hozKY4ZbUdTEZ9...,3,3
2,b'gAAAAABgSoAe3ZV1MseqV2GX8fwmdvj37D63Pig32vp3...,3,1
3,b'gAAAAABgSoAeULiizQNhiy-YFoaHsh63tKBHneY-yAZY...,1,3
4,b'gAAAAABgSoAeNwlxact0GfvRRmr7jm82I79TaR9NUGxU...,3,4
...,...,...,...
58,b'gAAAAABgSoAfyXH_zX-usQyNz0Nb70dAXisHOYxRekSJ...,2,1
59,b'gAAAAABgSoAfMD_705lv_4l6DbKZS-m0aCLYwvlpykX7...,2,1
60,b'gAAAAABgSoAftxP8B3OCpZhpuNIOv4qNoyUSKO1KNJaj...,3,1
61,b'gAAAAABgSoAfuIpl7qEHOhQ9KaV7m19IUe5b5eh4gW6j...,1,1


## Query Execution 1
Say we need to execute a query: 
```
select * from Adult where 25 < age < 30
```
Steps: 
* Consult the partition function and ask the server for all tuples with: AgeP = 4
* At the client, decrypt each of the tuples (client has the encryption key) and check which of the tuples actually satisfy the condition.

In [5]:
##### At the server
matching_tuples = encrypted_data[encrypted_data['AgeP'] == 4]
matching_tuples

,EncryptedTuple,ZipP,AgeP
4,b'gAAAAABgSoAeNwlxact0GfvRRmr7jm82I79TaR9NUGxU...,3,4
12,b'gAAAAABgSoAe6fHFLD_oCWUby-bmIN2kbwQN8NOpvS5x...,3,4
16,b'gAAAAABgSoAev1K-VOA2TFlmIMMEXAXRpdjAzd7ehYLx...,2,4
26,b'gAAAAABgSoAe7NLjB8-Pb-rI0Cc_S17P0TBj1TdwtNVv...,3,4
30,b'gAAAAABgSoAeCBhpKn8Tg_GBKcWdYn4uxQero6RVLIwJ...,3,4
31,b'gAAAAABgSoAezcGDIoJHEfGsNxLi4jJDYBtQUSbVZTH7...,3,4
34,b'gAAAAABgSoAem9XTXP0Jnb7q8LfBXDfnbGWrRxDPV_eo...,1,4
36,b'gAAAAABgSoAeK-3kmCq_3FlZrnd_amsVs-I_7XSg3u_7...,2,4
37,b'gAAAAABgSoAeVu8sr0GqhnMDYg81S2JynAarFk8WRvaG...,3,4
42,b'gAAAAABgSoAe-257afCCdL0kcDIjf-K3h6jf3g8vZLhX...,1,4


In [6]:
### At the client, we can decrypt all the results, however, we will probably get more results than needed
for index, row in matching_tuples.iterrows():
    
    decrypted_tuple = fernet_f.decrypt(row['EncryptedTuple']).decode('utf-8')
    print("====== Decrypted tuple {}".format(decrypted_tuple))

====== Decrypted tuple 75387;; 28;; Bachelors;; Married-civ-spouse;; Prof-specialty;; Black;; Female
====== Decrypted tuple 79482;; 23;; Bachelors;; Never-married;; Adm-clerical;; White;; Female
====== Decrypted tuple 32345;; 25;; HS-grad;; Never-married;; Farming-fishing;; White;; Male
====== Decrypted tuple 91102;; 19;; HS-grad;; Never-married;; Craft-repair;; White;; Male
====== Decrypted tuple 93239;; 23;; Bachelors;; Never-married;; Protective-serv;; White;; Male
====== Decrypted tuple 65932;; 20;; Some-college;; Never-married;; Sales;; Black;; Male
====== Decrypted tuple 20149;; 22;; Some-college;; Married-civ-spouse;; Other-service;; Black;; Male
====== Decrypted tuple 41136;; 21;; Some-college;; Never-married;; Machine-op-inspct;; White;; Male
====== Decrypted tuple 93276;; 19;; HS-grad;; Married-AF-spouse;; Adm-clerical;; White;; Female
====== Decrypted tuple 18173;; 24;; Bachelors;; Married-civ-spouse;; Tech-support;; White;; Male
====== Decrypted tuple 62177;; 25;; HS-grad;;

In [7]:
print("========= FINAL RESULT")
for index, row in matching_tuples.iterrows():
    decrypted_tuple = fernet_f.decrypt(row['EncryptedTuple']).decode('utf-8')
    if 25 < int(decrypted_tuple.split(";; ")[1]) < 30:
        print(decrypted_tuple)

========= FINAL RESULT
75387;; 28;; Bachelors;; Married-civ-spouse;; Prof-specialty;; Black;; Female
56298;; 29;; Masters;; Never-married;; Prof-specialty;; White;; Male


## Query Execution 2
What about
```
select * from Adult where 25 < age < 45
```
Now we need to look for: 
```
AgeP in [4, 1]
```

## Joins
Let's say we have another table: Generations(Age, GenerationName), using a different partition function, and we want to do a join between the two on Age

In [8]:
gens = pd.read_csv('generations.csv')
gens

,Age,Generation
0,1,Gen Alpha
1,2,Gen Alpha
2,3,Gen Alpha
3,4,Gen Alpha
4,5,Gen Alpha
...,...,...
106,107,Greatest Generation
107,108,Greatest Generation
108,109,Greatest Generation
109,110,Greatest Generation


In [9]:
def get_age_partition_gens_table(age):
    if age < 20: return 3
    if age < 60: return 4
    if age < 100: return 2
    return 1
fernet_f_g = Fernet(Fernet.generate_key())
encrypted_tuples_gens = {}
for index, row in gens.iterrows():
    str_row = ";; ".join([str(t) for t in tuple(row)])
    encrypted_tuples_gens[index] = [fernet_f_g.encrypt(str_row.encode('utf-8')), 
                                    get_age_partition_gens_table(row['Age'])]
encrypted_data_gens = pd.DataFrame.from_dict(encrypted_tuples_gens, orient='index', columns = ["EncryptedTuple", "AgeP"])
encrypted_data_gens

,EncryptedTuple,AgeP
0,b'gAAAAABgSoFYyPo35EDzaV9-7SgXq4Nb0r3C94rO6RS9...,3
1,b'gAAAAABgSoFY08m58ttj6oLjhiKP9KLQl2VxG10Do3tk...,3
2,b'gAAAAABgSoFYFOhk9sVgotGwlZos-Afv2_s3lS1WVh91...,3
3,b'gAAAAABgSoFY5rD95Br8j7k0AKyn_tZXN80yrOtM6DN-...,3
4,b'gAAAAABgSoFYYHOEGlFEniDTmmAgB6g_MSqijBZcU4hI...,3
...,...,...
106,b'gAAAAABgSoFYrX8ITOZfRNUZZa0YtRy9n56-l9r6rY-2...,1
107,b'gAAAAABgSoFYXXNKukELVbvurl3TEe7r_m0PawnAEUIo...,1
108,b'gAAAAABgSoFYLwQo0YE2ENMGsZbJBHTm76y_s6uOEOAN...,1
109,b'gAAAAABgSoFYlOZsnrcVgBtEKu9XCW6O5Lws3_nA1n76...,1


But now things are a little more complicated. 
A tuple in `Adult` matches a tuple in `Generations` if they have the same Age.
For example, the Adult tuple:
```
64152,39,Bachelors,Never-married,Adm-clerical,White,Male
```
matches Generations tuple:
```
39,Xennials
```
But for the first tuple, AgeP = 1, and for the second tuple, AgeP = 4

More generally, a tuple in `Adult` with AgeP = 1 (range: 30-49) joins with tuples from `Generations` with AgeP = 4 (range: 20-59).
`Adult` tuple with AgeP = 2 (range: 70-) may match with tuples from `Generations` with AgeP = 2 (60-99) or AgeP = 1 (100-).

So our join condition on `encrypted_data` and `encrypted_data_gens` on AgeP becomes:
```
(encrypted_data.AgeP = 1 AND encrypted_data_gens.AgeP = 4) OR
(encrypted_data.AgeP = 2 AND encrypted_data_gens.AgeP = 2) OR
(encrypted_data.AgeP = 2 AND encrypted_data_gens.AgeP = 1) OR
....
```

In [10]:
joined_tuples = []
for index1, row1 in encrypted_data.iterrows():
    for index2, row2 in encrypted_data_gens.iterrows():
        if (row1['AgeP'] == 1 and row2['AgeP'] == 4) or (False):
            joined_tuples = (row1, row2)
joined_tuples

(EncryptedTuple    b'gAAAAABgSoAfMMRsOlInKX4_oN-UbIO97mwaXjOJLq04...
 ZipP                                                              1
 AgeP                                                              1
 Name: 62, dtype: object,
 EncryptedTuple    b'gAAAAABgSoFYjOFkzZ_CoRRrd-BYj_Co8efeMrI4YC5s...
 AgeP                                                              4
 Name: 58, dtype: object)

## Search -- Simple Scheme

In [11]:
documents = ["I learn in this letter that Don Peter of Arragon comes this night to Messina.", 
            "A victory is twice itself when the achiever brings home full numbers. I find here that Don Peter hath bestowed much honour on a young Florentine called Claudio."]

In [22]:
encrypted_documents = [] 
fernet_f_d = Fernet(Fernet.generate_key())
for d in documents:
    e_kw = [fernet_f_d.encrypt(x.encode('utf-8')) for x in d.split()]
    encrypted_documents.append({"encrypted_doc": fernet_f_d.encrypt(d.encode('utf-8')), "associated_keywords": e_kw})

In [23]:
encrypted_documents

[{'encrypted_doc': b'gAAAAABgSoMYyKX1gmkD323RliAcVRpROwayvFhVZ06eey46d0Irv4lm3-RV3A30lhQ84ldjXGEjT8V6BzSVlAbUpJdcXFn3YN4CxvQk0NTEMugEp5hSnA49PXJqoGpwSU4vBVt7px-FcYMhM_NLgCaI-9-jB0HblGWB_CCDEDNSkewT2s1Vu20=',
  'associated_keywords': [b'gAAAAABgSoMYEODFaLlBE5pIaXwF3rndxLtmF2l-X8jUUECvaaVpWS25ZUYb4SzbMT3ugDh6mMo96a-W1NVA9bdOYkrGC4Insg==',
   b'gAAAAABgSoMY5SE-hUUfAyt4aMIhxoPz1dpj_O6hM77utk7Fyy1Xtgzr8n-7MdwIP06sNkVnYS7og9CQceLj11qN5lpGFiA6Gg==',
   b'gAAAAABgSoMYM_6kp7pSlgwjjguFDmw0-HH5rIDRqbGJQ1m4V-tRQ7j_p9cT3vee2k_ygePmranFMa9-ycaHj21-H2MwhJPqhg==',
   b'gAAAAABgSoMYpNjlEOqUKcjVm9zwqUO7vi6c8CG-_M7CMVqa1Rmy20ggAjy6861t-JfBgzymgM23tufXsVfg7fxdzwya3NtaRg==',
   b'gAAAAABgSoMYcVktDutp2LG9p7vSBUd7Jo3w1oPSzHzAuevufQ6cXR67CkE159HrIHzzsvDqqCytSsmV1AEIkG9NC029XGL9Yw==',
   b'gAAAAABgSoMY--lVOfaQBoBEmNkpdtd-UTQQscVvEcMj_IMhMAX33ls-REJ3FAVAcDcP_hjmGJu3aHrWQ6wvHTtGnd6ymA2PEw==',
   b'gAAAAABgSoMYwjlAOIPfXkt7ay_QWWUk0d0OmTzjDaK4mxt9KKQ4IrVfek4aUEwHsX8L6-_siYfBuygZ1OBUA1DSXDNTphvvQQ==',
   b'gAAAAABg

Let's say we want to search for a keyword 'Arragon'.
Steps: 
- We encrypt 'Arragon' and send it to the server
- The server looks for all documents that are associated with the keyword
- The client decrypts the documents and confirms that they contain the keyword

In [26]:
en_query = fernet_f_d.encrypt('Arragon'.encode('utf-8'))
en_query

b'gAAAAABgSoMkBgbihG0xrLesqEtYuoUUF9tGsB8esuRcX5P58gB38KFymYBq5R1JszLemuema5U80h7m7NpNyusLIpvdu6Wv2g=='

In [27]:
## Send to server who does a lookup
query_answer = [x['encrypted_doc'] for x in encrypted_documents if en_query in x['associated_keywords']]
query_answer

[]